In [17]:
from database import *

def ticks_to_kbars(ticks, interval='1Min'):

    kbars = pd.DataFrame()
    
    kbars['open'] = ticks['close'].resample(interval).first()
    kbars['close'] = ticks['close'].resample(interval).last()
    kbars['high'] = ticks['close'].resample(interval).max()
    kbars['low'] = ticks['close'].resample(interval).min()
    kbars['volume'] = ticks['volume'].resample(interval).sum()
    
    kbars.dropna(inplace=True)
    
    return kbars

In [19]:
import talib
def get_technical_indicator(kbars):
    
    kbars['rsi'] = talib.RSI(kbars.close, timeperiod=14)
    
    macd, macdsignal, macdhist = talib.MACD(kbars.close, fastperiod=12, slowperiod=26, signalperiod=9)
    
    kbars['macd'] = macd
    kbars['macdsignal'] = macdsignal
    kbars['macdhist'] = macdhist
    
    return kbars

In [20]:
api = sj.Shioaji()
api.login(
    person_id='N123569782',
    passwd='hons3146',
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.")
)

# api.logout()

connection = sqlite3.connect('data.db')

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done.
<SecurityType.Future: 'FUT'> fetch done.
<SecurityType.Stock: 'STK'> fetch done.
<SecurityType.Option: 'OPT'> fetch done.
Response Code: 0 | Event Code: 12 | Info: solClient.c:12194                    (000036e4) Session '(c2,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/N123569782/0802/102808/206576/111.82.227.147', VPN name 'sinopac', peer address 'hostname '203.66.91.161:80' IP 203. | Event: Session reconnecting
Response Code: 0 | Event Code: 13 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total reconnection attempt 1 of 10) | Event: Session reconnected
Response Code: 0 | Event Code: 12 | Info: solClient.c:12194        

In [8]:
code = '2441'

tw_calendar = get_calendar('XTAI')

date = pd.to_datetime('2021/8/2')
prev_trading_date = tw_calendar.previous_close(date).date()
    
ticks = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
kbars = ticks_to_kbars(ticks)
kbars = get_technical_indicator(kbars)
kbars = kbars[date:]

kbars

,open,close,high,low,volume,rsi,macd,macdsignal,macdhist
ts,,,,,,,,,
2021-08-02 09:00:00,96.4,96.0,96.5,95.8,174,57.355058,-0.112123,-0.172805,0.060682
2021-08-02 09:01:00,96.0,96.0,97.4,96.0,251,57.355058,-0.062770,-0.150798,0.088028
2021-08-02 09:02:00,96.1,95.8,96.1,95.5,134,53.069501,-0.039342,-0.128507,0.089165
2021-08-02 09:03:00,95.6,95.6,95.9,95.4,76,49.117168,-0.036492,-0.110104,0.073612
2021-08-02 09:04:00,95.4,95.6,95.6,95.3,80,49.117168,-0.033844,-0.094852,0.061008
...,...,...,...,...,...,...,...,...,...
2021-08-02 13:22:00,91.2,91.1,91.2,91.1,32,44.674803,0.009760,0.009040,0.000720
2021-08-02 13:23:00,91.1,91.0,91.1,91.0,98,41.776072,-0.011904,0.004851,-0.016755
2021-08-02 13:24:00,91.0,91.2,91.6,91.0,82,48.915294,-0.012787,0.001323,-0.014110


In [15]:
def day_trading_backtest(code, date, connection, api):
    
    tw_calendar = get_calendar('XTAI')
    prev_trading_date = tw_calendar.previous_close(date).date()
    
    ticks = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
    kbars = ticks_to_kbars(ticks)
    kbars = get_technical_indicator(kbars)
    kbars = kbars[date:]
            
    entry_price = 0
    entry_time = None
    
    exit_price = 0
    exit_time = None
    
    position = 0
            
    for ts in range(len(kbars)):
        
        current_time = kbars.iloc[ts].name + pd.Timedelta(minutes=1)
        current_price = kbars['close'][ts]
        
        if (
            kbars.iloc[ts - 3:ts]['rsi'].min() < 30 and
            kbars.iloc[ts - 1]['macdhist'] < 0 and
            kbars.iloc[ts]['macdhist'] > 0 and
            position == 0
        ):
            
            position = 1          
            entry_price = current_price
            entry_time = current_time.time()

            print('[{}] buy {} at {}'.format(current_time, code, current_price))
        
        elif (
            kbars.iloc[ts - 3:ts]['rsi'].max() > 70 and
            kbars.iloc[ts - 1]['macdhist'] > 0 and
            kbars.iloc[ts]['macdhist'] < 0 and
            position != 0
        ):
            exit_price = current_price
            exit_time = current_time.time()
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
        elif (
            current_time == date.replace(hour=13, minute=0, second=0) and
            position != 0
        ):
            
            exit_price = current_price
            exit_time = current_time.time()
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
    if entry_time and exit_time:
        transaction = pd.DataFrame([[date, 
                                     code, 
                                     entry_time,
                                     entry_price, 
                                     position * 1000,
                                     entry_price * position * 1000,
                                     exit_time,
                                     exit_price,
                                     position * 1000,
                                     exit_price * position * 1000]
                                   ],
                                    columns=[
                                        '成交日期', 
                                        '股票代號',
                                        '買進時間',
                                        '買進價格',
                                        '買進股數',
                                        '買進金額',
                                        '賣出時間',
                                        '賣出價格',
                                        '賣出股數',
                                        '賣出金額'])
        return transaction
    else:
        return pd.DataFrame()

In [16]:
day_trading_backtest('2303', pd.to_datetime('2021/8/2'), connection, api)

[2021-08-02 12:10:00] buy 2303 at 57.5
[2021-08-02 13:00:00] sell 2303 at 57.6


,成交日期,股票代號,買進時間,買進價格,買進股數,買進金額,賣出時間,賣出價格,賣出股數,賣出金額
0,2021-08-02,2303,12:10:00,57.5,1000,57500.0,13:00:00,57.6,1000,57600.0
